In [ ]:
import asyncio
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import SelectorGroupChat
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.conditions import TextMentionTermination, MaxMessageTermination
from dotenv import load_dotenv
import os

load_dotenv()


In [ ]:
# api_key = os.getenv('GOOGLE_API_KEY')
# model_client = OpenAIChatCompletionClient(model='gemini-2.5-flash', api_key=api_key)

In [17]:
api_key = os.getenv('OPENAI_API_KEY')
model_client = OpenAIChatCompletionClient(model='gpt-4o', api_key=api_key)

In [3]:
planning_agent = AssistantAgent(
    name="PlanningAgent",
    description="An agent for planning tasks, this agent should be the first to engage when given a new task.",
    model_client=model_client,
    system_message="""
    You are a planning agent.
    Your job is to break down complex tasks into smaller, manageable subtasks.
    Your team members are:
        WebSearchAgent: Searches for information
        DataAnalystAgent: Performs calculations

    You only plan and delegate tasks - you do not execute them yourself.

    When assigning tasks, use this format:
    1. <agent> : <task>

    After all tasks are complete, summarize the findings and end with "TERMINATE".
    """,
)

In [4]:
from langchain_community.utilities import GoogleSerperAPIWrapper
from autogen_ext.tools.http import HttpTool
os.environ['SERPER_API_KEY']='bead05022450578faa7498f4c90d85e534c372e0'

In [5]:
search_tool_wrapper = GoogleSerperAPIWrapper(type='search')

def search_web(query:str) ->str:
    """Search the web for the given query and return the results."""
    try:
        results = search_tool_wrapper.run(query)
        return results
    except Exception as e:
        print(f"Error occurred while searching the web: {e}")
        return "No results found."  

In [6]:
def search_web_tool(query:str)-> str:

    if "2006-2007" in query:
        return """Here are the total points scored by Miami Heat players in the 2006-2007 season:
        Udonis Haslem: 844 points
        Dwayne Wade: 1397 points
        James Posey: 550 points
        ...
        """
    elif "2007-2008" in query:
        return "The number of total rebounds for Dwayne Wade in the Miami Heat season 2007-2008 is 214."
    elif "2008-2009" in query:
        return "The number of total rebounds for Dwayne Wade in the Miami Heat season 2008-2009 is 398."
    return "No data found."

In [18]:
web_search_agent = AssistantAgent(
    name = 'WebSearchAgent',
    description= 'An agent for searching the web for information.',
    model_client=model_client,
    tools = [search_web_tool],
    reflect_on_tool_use=False,
    system_message='''
        You are a web search agent.
        Your only tool is search_web - use it to find the information you need.

        You make only one search call at a time.
        
        Once you have the results, you never do calculations or data analysis on them.
    ''',
)

In [19]:
def percentage_change_tool(a:float, b:float) -> float:
    if a == 0:
        return 0
    return ((b - a) / a) * 100

In [20]:
data_analyst_agent = AssistantAgent(
    name = 'DataAnalystAgent',
    description= 'An agent for performing calculations and data analysis.',
    model_client=model_client,
    tools= [percentage_change_tool],
    system_message='''
        You are a data analyst agent.
        Given the tasks you have been assigned, you should analyze the data and provide results using the tools provided (percentage_change_tool).

        If you have not seen the data, ask for it.

    '''
)

In [21]:
text_mention_termination = TextMentionTermination('TERMINATE')
max_message_termination = MaxMessageTermination(max_messages=20)

combined_termination = text_mention_termination | max_message_termination

In [22]:
selector_prompt = '''
    Select an agent to perform the task.

    {roles}

    current conversation history :
    {history}

    Read the above conversation, then select an agent from {participants} to perform the next task.
    Make sure that the planning agent has assigned task before other agents start working.
    Only select one agent.
''' 

In [23]:
planning_agent.description

'An agent for planning tasks, this agent should be the first to engage when given a new task.'

In [24]:
selector_team = SelectorGroupChat(
    participants=[planning_agent, web_search_agent, data_analyst_agent],
    model_client=model_client,
    termination_condition=combined_termination,
    selector_prompt=selector_prompt,
    allow_repeated_speaker=True
)

In [25]:
task = "Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?"

In [26]:
from autogen_agentchat.ui import Console

await Console(selector_team.run_stream(task=task))

---------- TextMessage (user) ----------
Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?
---------- TextMessage (PlanningAgent) ----------
1. DataAnalystAgent: Identify the player with the highest points in the 2006-2007 season from the provided data.
2. DataAnalystAgent: Calculate the percentage change in total rebounds for Dwyane Wade between the 2007-2008 (214 rebounds) and 2008-2009 (398.8598130841121495 rebounds) seasons using the formula: ((Rebounds 2008-2009 - Rebounds 2007-2008) / Rebounds 2007-2008) * 100.
---------- TextMessage (DataAnalystAgent) ----------
Please provide the data for the Miami Heat player statistics for the 2006-2007 season so that I can identify the player with the highest points.
---------- ToolCallRequestEvent (WebSearchAgent) ----------
[FunctionCall(id='call_ofXucWF9yf12SYKj1ev2v0O7', arguments='{"query":"Miami Heat 2006-20

TaskResult(messages=[TextMessage(id='63f25e38-e422-475c-894a-f405eae75ce2', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 8, 21, 7, 15, 39, 526617, tzinfo=datetime.timezone.utc), content='Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?', type='TextMessage'), TextMessage(id='bf8980a9-af2d-4517-bce9-fc01cad71585', source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=537, completion_tokens=156), metadata={}, created_at=datetime.datetime(2025, 8, 21, 7, 15, 45, 279786, tzinfo=datetime.timezone.utc), content='1. DataAnalystAgent: Identify the player with the highest points in the 2006-2007 season from the provided data.\n2. DataAnalystAgent: Calculate the percentage change in total rebounds for Dwyane Wade between the 2007-2008 (214 rebounds) and 2008-2009 (398.8598130841121495 rebounds) seasons using the formula: 

In [ ]:
state = await selector_team.save_state()

In [36]:
import json
with open('state.json',mode='w') as f:
    json.dump(state,f)

In [41]:
answer = state["agent_states"]["SelectorGroupChatManager"]["message_thread"][-1]["content"].replace('TERMINATE', '')
answer

"The Miami Heat player with the highest points in the 2006-2007 season, based on the provided data, was Dwyane Wade with 1397 points.\n\nThe percentage change in Dwyane Wade's total rebounds between the 2007-2008 and 2008-2009 seasons was approximately 86.38%.\n\n"